In [ ]:
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import cv2
import tensorflow.compat.v1 as tf
import skimage.color as color
import skimage.io as io
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt


In [ ]:
mydir ="/kaggle/input/sample/"
N= 1
images = [files for files in os.listdir(mydir)][1:2]
print(type(images))
print(len(images))

data = np.zeros([N, 256, 256, 3]) #image loaded in this array
for count in range(N):
    img = cv2.resize(io.imread(mydir + '/'+ images[count]), (256, 256))
    grey = cv2.resize(cv2.imread(mydir + '/'+ images[count],0),(256,256))
    data[count,:,:,:] = img
print("grayscale image")
plt.imshow(grey) #saving the input image
plt.show()
num_train = N
Xtrain = color.rgb2lab(data[:num_train]*1.0/255)
xt = Xtrain[:,:,:,0] #loading l channel as input
yt = Xtrain[:,:,:,1:] #loading ab channel
yt = yt/128 #normalization
xt = xt.reshape(num_train, 256, 256, 1) #resizing inpput
yt = yt.reshape(num_train, 256, 256, 2) #resizing output

In [ ]:
tf.disable_eager_execution()
session = tf.Session()
x = tf.placeholder(tf.float32, shape = [None, 256, 256, 1], name = 'x')
ytrue = tf.placeholder(tf.float32, shape = [None, 256, 256, 2], name = 'ytrue')

In [ ]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def create_bias(size):
    return tf.Variable(tf.constant(0.1, shape = [size]))

In [ ]:
def convolution(inputs, num_channels, filter_size, num_filters):
    weights = create_weights(shape = [filter_size, filter_size, num_channels, num_filters])
    bias = create_bias(num_filters)
    layer = tf.nn.conv2d(input = inputs, filter = weights, strides= [1, 1, 1, 1], padding = 'SAME') + bias
    layer = tf.nn.tanh(layer)
    return layer

In [ ]:
def maxpool(inputs, kernel, stride):
    layer = tf.nn.max_pool(value = inputs, ksize = [1, kernel, kernel, 1], strides = [1, stride, stride, 1], padding = "SAME")
    return layer
def upsampling(inputs):
    layer = tf.image.resize_nearest_neighbor(inputs, (2*inputs.get_shape().as_list()[1], 2*inputs.get_shape().as_list()[2]))
    return layer

In [ ]:
# model layers
conv1 = convolution(x, 1, 3, 3)
max1 = maxpool(conv1, 2, 2)
conv2 = convolution(max1, 3, 3, 8)
max2 = maxpool(conv2, 2, 2)
conv3 = convolution(max2, 8, 3, 16)
max3 = maxpool(conv3, 2, 2)
conv4 = convolution(max3, 16, 3, 16)
max4 = maxpool(conv4, 2, 2)
conv5 = convolution(max4, 16, 3, 32)
max5 = maxpool(conv5, 2, 2)
conv6 = convolution(max5, 32, 3, 32)
max6 = maxpool(conv6, 2, 2)
conv7 = convolution(max6, 32, 3, 64)
upsample1 = upsampling(conv7)
conv8 = convolution(upsample1, 64, 3, 32)
upsample2 = upsampling(conv8)
conv9 = convolution(upsample2, 32, 3, 32)
upsample3 = upsampling(conv9)
conv10 = convolution(upsample3, 32, 3, 16)
upsample4 = upsampling(conv10)
conv11 = convolution(upsample4, 16, 3, 16)
upsample5 = upsampling(conv11)
conv12 = convolution(upsample5, 16, 3, 8)
upsample6 = upsampling(conv12)
conv13 = convolution(upsample6, 8, 3, 2)


In [ ]:
loss = tf.losses.mean_squared_error(labels = ytrue, predictions = conv13)
cost = tf.reduce_mean(loss)
# optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(cost)
# session.run(tf.global_variables_initializer())

In [ ]:
# saver = tf.train.Saver() #//train

with tf.Session() as session:
    new_saver = tf.train.import_meta_graph('/kaggle/input/lasta6/a6.meta')
    new_saver.restore(session, tf.train.latest_checkpoint('/kaggle/input/lasta6/'))
#     num_epochs = 50
#     for i in range(num_epochs):
#         session.run(conv13, feed_dict = {x: xt, ytrue:yt})
#         lossvalue = session.run(cost, feed_dict = {x:xt, ytrue : yt})
#         print("epoch: " + str(i) + " loss: " + str(lossvalue))
#     saver.save(session, 'train1') #//train
    
    #using trained model
    output = session.run(conv13, feed_dict = {x: xt[0].reshape([1, 256, 256, 1])})*128
#     output = session.run( )

    image = np.zeros([256, 256, 3])
    image[:,:,0]=xt[0][:,:,0]
    image[:,:,1:]=output[0]
    image = color.lab2rgb(image)
    plt.imshow(image)
    plt.show()